# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,2.51,98,100,0.65,RU,1694865283
1,1,waitangi,-43.9535,-176.5597,11.01,84,97,3.13,NZ,1694865283
2,2,yabelo,4.8833,38.0833,23.44,40,65,2.43,ET,1694865284
3,3,isafjordur,66.0755,-23.1240,9.02,66,75,5.14,IS,1694865284
4,4,west island,-12.1568,96.8225,25.99,73,40,6.69,CC,1694865284


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
                                       & (city_data_df["Max Temp"] > 21) 
                                       & (city_data_df["Cloudiness"] < 3)
                                       & (city_data_df["Wind Speed"] < 4.5)
                                      ]



# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
244,244,broken hill,-31.9500,141.4333,22.62,21,0,3.83,AU,1694865334
255,255,ciudad insurgentes,25.2625,-111.7753,24.79,60,2,3.67,MX,1694865336
288,288,nelson bay,-32.7167,152.1500,21.11,53,0,1.79,AU,1694865342
289,289,galliano,29.4422,-90.2992,23.33,94,0,0.00,US,1694865342
295,295,humenne,48.9371,21.9163,25.03,44,1,2.33,SK,1694865343
297,297,rio brilhante,-21.8019,-54.5464,23.41,37,0,3.10,BR,1694865343
310,310,tefe,-3.3542,-64.7114,25.06,94,0,1.03,BR,1694865345
311,311,ribas do rio pardo,-20.4431,-53.7592,25.63,38,0,3.26,BR,1694865345
374,374,puerto carreno,6.1890,-67.4859,25.99,94,0,0.00,CO,1694865358
397,397,villa union,23.2000,-106.2333,25.19,91,2,1.61,MX,1694865363


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
244,broken hill,AU,-31.9500,141.4333,21,
255,ciudad insurgentes,MX,25.2625,-111.7753,60,
288,nelson bay,AU,-32.7167,152.1500,53,
289,galliano,US,29.4422,-90.2992,94,
295,humenne,SK,48.9371,21.9163,44,
297,rio brilhante,BR,-21.8019,-54.5464,37,
310,tefe,BR,-3.3542,-64.7114,94,
311,ribas do rio pardo,BR,-20.4431,-53.7592,38,
374,puerto carreno,CO,6.1890,-67.4859,94,
397,villa union,MX,23.2000,-106.2333,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
broken hill - nearest hotel: Ibis Styles
ciudad insurgentes - nearest hotel: Hotel Restaurant Californias
nelson bay - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
humenne - nearest hotel: Alibaba
rio brilhante - nearest hotel: Gold Hotel
tefe - nearest hotel: Pousada Santa Teresa
ribas do rio pardo - nearest hotel: Hotel São Francisco
puerto carreno - nearest hotel: Carreño Plaza
villa union - nearest hotel: No hotel found
moapa valley - nearest hotel: No hotel found
chulakkurgan - nearest hotel: адеми
robore - nearest hotel: Lajas


,City,Country,Lat,Lng,Humidity,Hotel Name
244,broken hill,AU,-31.9500,141.4333,21,Ibis Styles
255,ciudad insurgentes,MX,25.2625,-111.7753,60,Hotel Restaurant Californias
288,nelson bay,AU,-32.7167,152.1500,53,No hotel found
289,galliano,US,29.4422,-90.2992,94,Fairfield Inn & Suites by Marriott Cut Off-Gal...
295,humenne,SK,48.9371,21.9163,44,Alibaba
297,rio brilhante,BR,-21.8019,-54.5464,37,Gold Hotel
310,tefe,BR,-3.3542,-64.7114,94,Pousada Santa Teresa
311,ribas do rio pardo,BR,-20.4431,-53.7592,38,Hotel São Francisco
374,puerto carreno,CO,6.1890,-67.4859,94,Carreño Plaza
397,villa union,MX,23.2000,-106.2333,91,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)